In [1]:
import itertools
import numpy as np
from dataclasses import dataclass
from typing import Tuple, List, Optional

from scipy.optimize import linprog, minimize
from scipy.sparse import coo_matrix, csr_matrix


# --------------------------
# Utilities
# --------------------------

def all_bitstrings(n: int) -> np.ndarray:
    return np.array(list(itertools.product([0, 1], repeat=n)), dtype=int)


def all_branches(n: int, h: int) -> List[Tuple[Tuple[int, ...], Tuple[int, ...]]]:
    k = n - h
    branches = []
    for A in itertools.combinations(range(n), k):
        for aA in itertools.product([0, 1], repeat=k):
            branches.append((tuple(A), tuple(aA)))
    return branches


def sigmoid(z):
    z = np.clip(z, -40, 40)
    return 1.0 / (1.0 + np.exp(-z))


def sort_desc(s: np.ndarray) -> np.ndarray:
    return np.sort(np.asarray(s, dtype=float))[::-1]


def linprog_any(c, A_ub, b_ub, A_eq, b_eq, bounds):
    methods = ("highs", "highs-ds", "highs-ipm", "interior-point", "revised simplex", "simplex")
    last_err = None
    for method in methods:
        try:
            return linprog(
                c=c, A_ub=A_ub, b_ub=b_ub,
                A_eq=A_eq, b_eq=b_eq,
                bounds=bounds,
                method=method
            )
        except ValueError as e:
            last_err = e
            continue
    if last_err is not None:
        raise last_err
    raise RuntimeError("linprog failed unexpectedly.")


def cache_key_from_s(s: np.ndarray, *, tol: float = 1e-4) -> tuple:
    s = np.clip(np.asarray(s, dtype=float), 0.0, 1.0)
    q = np.round(s / tol).astype(np.int64)
    diffs = np.diff(q)
    return (int(q[0]), *map(int, diffs))


# --------------------------
# Inner LP solver
# --------------------------

@dataclass
class LPValue:
    ok: bool
    t: float
    f: Optional[np.ndarray]
    msg: str


class FullProgramLPSolver:
    """
    Fixes the build_ub bottleneck:
      - precompute A_ub CSR structure ONCE
      - per solve, only fill COO-order data vector then permute into CSR-order data
      - reuse indices/indptr (no COO->CSR conversion per miss)

    Also uses cheap weight computation from term = where(bits, s, 1-s):
      - pi = prod(term, axis=1)
      - eq weights: w_i = pi / term[:, i]
      - branch weights: w_A = prod(term[:, comp(A)], axis=1)   where comp(A) has size h
    """

    def __init__(
        self,
        n: int,
        h: int,
        C: float,
        cache_tol: float = 1e-4,
        cache_max: int = 200_000,
        profile: bool = True,
    ):
        self.n = int(n)
        self.h = int(h)
        self.C = float(C)

        self.bits = all_bitstrings(self.n)  # (m,n)
        self.m = self.bits.shape[0]
        self.all_zero = (self.bits.sum(axis=1) == 0).astype(float)  # (m,)

        self.branches = all_branches(self.n, self.h)
        self.num_branches = len(self.branches)

        # Variable layout
        self.N = self.n * self.m + 1
        self.t_idx = self.n * self.m
        self.bounds = [(0.0, None)] * (self.n * self.m) + [(0.0, None)]
        self.c = np.zeros(self.N, dtype=float)
        self.c[self.t_idx] = 1.0

        # Precompute eq index sets
        self.eq_idx1 = [np.where(self.bits[:, i] == 1)[0] for i in range(self.n)]
        self.eq_idx0 = [np.where(self.bits[:, i] == 0)[0] for i in range(self.n)]
        self.eq_cols1 = [i * self.m + self.eq_idx1[i] for i in range(self.n)]
        self.eq_cols0 = [i * self.m + self.eq_idx0[i] for i in range(self.n)]

        # Branch precompute:
        # - idx of a matching (A,aA)
        # - comp(A) columns (size h)
        # - whether idx contains 0 (all-zero bitstring, which is row 0)
        all_idx = np.arange(self.n)
        self.branch_match_idx: List[np.ndarray] = []
        self.branch_cols_per_i: List[List[np.ndarray]] = []
        self.branch_comp_cols: List[np.ndarray] = []
        self.branch_has_zero = np.zeros(self.num_branches, dtype=bool)

        for b, (A, aA) in enumerate(self.branches):
            mask = np.ones(self.m, dtype=bool)
            for pos, j in enumerate(A):
                mask &= (self.bits[:, j] == aA[pos])
            idx = np.where(mask)[0]
            self.branch_match_idx.append(idx)
            self.branch_cols_per_i.append([i * self.m + idx for i in range(self.n)])

            Aset = set(A)
            comp = np.array([j for j in all_idx if j not in Aset], dtype=int)  # size h
            self.branch_comp_cols.append(comp)

            self.branch_has_zero[b] = (idx.size > 0 and idx[0] == 0) or (0 in set(idx.tolist()))

        # ----------------------
        # Precompute A_ub pattern in COO order (rows, cols) + slices
        # ----------------------
        self.num_ub = 1 + self.num_branches

        total_nnz = self.n * self.m
        for idx in self.branch_match_idx:
            total_nnz += self.n * idx.size + 1

        self._ub_rows = np.empty(total_nnz, dtype=int)
        self._ub_cols = np.empty(total_nnz, dtype=int)

        self._ir_slices: List[slice] = []
        self._branch_slices: List[List[slice]] = []
        self._t_pos = np.empty(self.num_branches, dtype=int)

        off = 0
        # IR row pattern (row 0)
        for i in range(self.n):
            cols = i * self.m + np.arange(self.m, dtype=int)
            self._ub_rows[off:off + self.m] = 0
            self._ub_cols[off:off + self.m] = cols
            self._ir_slices.append(slice(off, off + self.m))
            off += self.m

        # Branch row patterns
        for b in range(self.num_branches):
            row = 1 + b
            idx = self.branch_match_idx[b]
            b_slices = []
            for i in range(self.n):
                cols = self.branch_cols_per_i[b][i]
                L = cols.size
                self._ub_rows[off:off + L] = row
                self._ub_cols[off:off + L] = cols
                b_slices.append(slice(off, off + L))
                off += L

            # -t entry
            self._ub_rows[off] = row
            self._ub_cols[off] = self.t_idx
            self._t_pos[b] = off
            off += 1
            self._branch_slices.append(b_slices)

        assert off == total_nnz

        # ----------------------
        # Build CSR template ONCE + permutation mapping COO-order -> CSR-order
        # Trick: put data = 0..nnz-1, convert to CSR, then csr.data are those ids in CSR order.
        # ----------------------
        ids = np.arange(total_nnz, dtype=float)
        csr_template = coo_matrix((ids, (self._ub_rows, self._ub_cols)),
                                  shape=(self.num_ub, self.N)).tocsr()

        # perm[k] = which COO entry sits at CSR position k
        self._ub_perm = csr_template.data.astype(np.int64)
        self._ub_indices = csr_template.indices.copy()
        self._ub_indptr = csr_template.indptr.copy()

        # A_eq is tiny; keep building it per miss (it wasn't your bottleneck)

        # Cache
        self.cache_tol = float(cache_tol)
        self._cache: dict[tuple, LPValue] = {}
        self._cache_max = int(cache_max)

        # Profiling
        self.profile = bool(profile)
        self._prof = {
            "calls": 0,
            "cache_hits": 0,
            "build_eq_s": 0.0,
            "build_ub_s": 0.0,
            "linprog_s": 0.0,
            "total_s": 0.0,
        }

    def solve_lp(self, s: np.ndarray, return_f: bool = False) -> LPValue:
        import time
        t0 = time.perf_counter()
        if self.profile:
            self._prof["calls"] += 1

        s = np.clip(np.asarray(s, dtype=float), 0.0, 1.0)

        key = cache_key_from_s(s, tol=self.cache_tol)
        cached = self._cache.get(key)
        if cached is not None and ((not return_f) or (cached.f is not None)):
            if self.profile:
                self._prof["cache_hits"] += 1
                self._prof["total_s"] += time.perf_counter() - t0
            return cached

        n, m, C = self.n, self.m, self.C
        N = self.N

        # Precompute term + pi once
        s_safe = np.clip(s, 1e-12, 1 - 1e-12)
        term = np.where(self.bits == 1, s_safe[None, :], (1.0 - s_safe)[None, :])  # (m,n)
        pi = term.prod(axis=1)  # (m,)

        # ----------------------
        # Build A_eq (sparse)
        # ----------------------
        t_eq0 = time.perf_counter()
        eq_rows = []
        eq_cols = []
        eq_data = []

        for i in range(n):
            w_i = pi / term[:, i]
            idx1 = self.eq_idx1[i]
            idx0 = self.eq_idx0[i]
            cols1 = self.eq_cols1[i]
            cols0 = self.eq_cols0[i]

            eq_rows.append(np.full(cols1.shape[0], i, dtype=int))
            eq_cols.append(cols1.astype(int))
            eq_data.append(w_i[idx1])

            eq_rows.append(np.full(cols0.shape[0], i, dtype=int))
            eq_cols.append(cols0.astype(int))
            eq_data.append(-w_i[idx0])

        A_eq = coo_matrix(
            (np.concatenate(eq_data), (np.concatenate(eq_rows), np.concatenate(eq_cols))),
            shape=(n, N),
        ).tocsr()
        b_eq = np.ones(n, dtype=float)

        if self.profile:
            self._prof["build_eq_s"] += time.perf_counter() - t_eq0

        # ----------------------
        # Build A_ub by ONLY filling data (no COO->CSR conversion)
        # ----------------------
        t_ub0 = time.perf_counter()

        data_coo = np.empty_like(self._ub_rows, dtype=float)
        b_ub = np.zeros(self.num_ub, dtype=float)

        # IR row blocks: fill -pi
        for sl in self._ir_slices:
            data_coo[sl] = -pi
        b_ub[0] = -float(s.sum())

        # Branch rows
        for b in range(self.num_branches):
            comp = self.branch_comp_cols[b]  # size h
            if comp.size == 0:
                w = np.ones(m, dtype=float)
            else:
                w = term[:, comp].prod(axis=1)  # (m,)

            idx = self.branch_match_idx[b]
            w_idx = w[idx]

            for i in range(n):
                data_coo[self._branch_slices[b][i]] = w_idx

            data_coo[self._t_pos[b]] = -1.0

            # RHS constant uses only the all-zero assignment (bitstring 000...0 is index 0)
            if self.branch_has_zero[b]:
                b_ub[1 + b] = -C * float(w[0])
            else:
                b_ub[1 + b] = 0.0

        # Reorder data into CSR order using perm computed once
        data_csr = data_coo[self._ub_perm]

        A_ub = csr_matrix((data_csr, self._ub_indices, self._ub_indptr), shape=(self.num_ub, self.N))

        if self.profile:
            self._prof["build_ub_s"] += time.perf_counter() - t_ub0

        # ----------------------
        # Solve LP
        # ----------------------
        t_lp0 = time.perf_counter()
        res = linprog_any(self.c, A_ub, b_ub, A_eq, b_eq, self.bounds)
        if self.profile:
            self._prof["linprog_s"] += time.perf_counter() - t_lp0

        if not res.success:
            val = LPValue(ok=False, t=float("inf"), f=None, msg=str(res.message))
            if len(self._cache) < self._cache_max:
                self._cache[key] = val
            if self.profile:
                self._prof["total_s"] += time.perf_counter() - t0
            return val

        t_val = float(res.fun)
        if not return_f:
            val = LPValue(ok=True, t=t_val, f=None, msg="OK")
            if len(self._cache) < self._cache_max:
                self._cache[key] = val
            if self.profile:
                self._prof["total_s"] += time.perf_counter() - t0
            return val

        x = res.x
        f = x[: n * m].reshape((n, m))
        val = LPValue(ok=True, t=t_val, f=f, msg="OK")
        if len(self._cache) < self._cache_max:
            self._cache[key] = val
        if self.profile:
            self._prof["total_s"] += time.perf_counter() - t0
        return val

    def print_profile(self, label: str = ""):
        p = self._prof
        calls = p["calls"]
        hits = p["cache_hits"]
        misses = calls - hits
        head = f"=== solve_lp profile {label} ===".strip()
        print("\n" + head)
        print(f"calls:      {calls}")
        print(f"cache_hits: {hits} ({hits / max(1, calls):.1%})")
        print(f"misses:     {misses} ({misses / max(1, calls):.1%})")
        print(f"build_eq:   {p['build_eq_s']:.3f}s")
        print(f"build_ub:   {p['build_ub_s']:.3f}s")
        print(f"linprog:    {p['linprog_s']:.3f}s")
        print(f"total:      {p['total_s']:.3f}s")


# --------------------------
# Outer optimization
# --------------------------

def blocks_from_equalities(n: int, tight_adj: Tuple[int, ...]) -> List[List[int]]:
    tight = set(tight_adj)
    blocks = []
    cur = [0]
    for i in range(n - 1):
        if i in tight:
            cur.append(i + 1)
        else:
            blocks.append(cur)
            cur = [i + 1]
    blocks.append(cur)
    return blocks


def enumerate_faces(n: int, include_endpoints: bool = True):
    faces = []
    for mask in range(1 << (n - 1)):
        tight_adj = tuple(i for i in range(n - 1) if (mask >> i) & 1)
        if include_endpoints:
            for top_fixed in (False, True):
                for bottom_fixed in (False, True):
                    faces.append((tight_adj, top_fixed, bottom_fixed))
        else:
            faces.append((tight_adj, False, False))
    return faces


def face_parameterization(z: np.ndarray, k: int, top_fixed: bool, bottom_fixed: bool) -> np.ndarray:
    z = np.asarray(z, dtype=float)
    v = np.zeros(k, dtype=float)
    if k == 0:
        return v

    idx = 0
    if top_fixed:
        v[0] = 1.0
    else:
        v[0] = sigmoid(z[idx]); idx += 1

    last_free = k - (1 if bottom_fixed else 0)
    for j in range(1, last_free):
        v[j] = v[j - 1] * sigmoid(z[idx])
        idx += 1

    if bottom_fixed:
        v[-1] = 0.0

    return v


def blocks_to_full_s(n: int, blocks: List[List[int]], v: np.ndarray) -> np.ndarray:
    s = np.zeros(n, dtype=float)
    for b_idx, idxs in enumerate(blocks):
        s[idxs] = v[b_idx]
    return s


@dataclass
class SearchResult:
    ok: bool
    t: float
    s: np.ndarray
    face: Optional[Tuple[Tuple[int, ...], bool, bool]]
    msg: str


def minimize_over_s_sorted(
    solver: FullProgramLPSolver,
    n_random: int = 120,
    n_local_starts: int = 3,
    seed: int = 0,
    local_method: str = "Powell",
    maxiter: int = 250,
) -> SearchResult:
    n = solver.n
    rng = np.random.default_rng(seed)

    best_t = float("inf")
    best_s = None

    for _ in range(n_random):
        s = sort_desc(rng.random(n))
        val = solver.solve_lp(s, return_f=False)
        if val.ok and val.t < best_t:
            best_t = val.t
            best_s = s.copy()

    if best_s is None:
        best_s = np.full(n, 0.5)

    def obj(z):
        s = sort_desc(sigmoid(z))
        val = solver.solve_lp(s, return_f=False)
        return val.t if val.ok else 1e6

    def inv_sig(u):
        u = np.clip(u, 1e-6, 1 - 1e-6)
        return np.log(u / (1 - u))

    starts = [inv_sig(best_s)]
    for _ in range(max(0, n_local_starts - 1)):
        s0 = sort_desc(rng.random(n))
        starts.append(inv_sig(s0))

    best_z = None
    best_local = best_t

    for z0 in starts:
        res = minimize(obj, z0, method=local_method, options={"maxiter": maxiter, "disp": False})
        if float(res.fun) < best_local:
            best_local = float(res.fun)
            best_z = res.x.copy()

    s_star = sort_desc(sigmoid(best_z)) if best_z is not None else best_s
    lp_star = solver.solve_lp(s_star, return_f=False)
    return SearchResult(lp_star.ok, lp_star.t, s_star, None, lp_star.msg)


def minimize_over_faces(
    solver: FullProgramLPSolver,
    seed: int = 0,
    local_method: str = "Powell",
    n_local_starts_per_face: int = 1,
    include_endpoints: bool = True,
    max_faces: Optional[int] = None,
    maxiter: int = 120,
) -> SearchResult:
    n = solver.n
    rng = np.random.default_rng(seed)

    faces = enumerate_faces(n, include_endpoints=include_endpoints)
    if max_faces is not None:
        faces = faces[:max_faces]

    best = SearchResult(False, float("inf"), np.zeros(n), None, "no feasible face found")

    for (tight_adj, top_fixed, bottom_fixed) in faces:
        blocks = blocks_from_equalities(n, tight_adj)
        k = len(blocks)

        if k == 1 and top_fixed and bottom_fixed:
            continue

        dim = 0
        if not top_fixed:
            dim += 1
        dim += max(0, (k - 1) - (1 if bottom_fixed else 0))

        if dim == 0:
            v = np.zeros(k)
            if top_fixed:
                v[0] = 1.0
            if bottom_fixed:
                v[-1] = 0.0
            s = blocks_to_full_s(n, blocks, v)
            val = solver.solve_lp(s, return_f=False)
            if val.ok and val.t < best.t:
                best = SearchResult(True, val.t, s, (tight_adj, top_fixed, bottom_fixed), "OK")
            continue

        def obj(z):
            v = face_parameterization(z, k, top_fixed, bottom_fixed)
            s = blocks_to_full_s(n, blocks, v)
            val = solver.solve_lp(s, return_f=False)
            return val.t if val.ok else 1e6

        for _ in range(n_local_starts_per_face):
            z0 = rng.normal(size=dim)
            res = minimize(obj, z0, method=local_method, options={"maxiter": maxiter, "disp": False})
            if float(res.fun) < best.t:
                v = face_parameterization(res.x, k, top_fixed, bottom_fixed)
                s = blocks_to_full_s(n, blocks, v)
                best = SearchResult(True, float(res.fun), s, (tight_adj, top_fixed, bottom_fixed), "OK")

    return best

In [2]:
n = 5
C = 11.0

interior_n_random = 120
interior_n_local_starts = 3
interior_maxiter = 250

face_local_starts_per_face = 1
face_include_endpoints = True
face_max_faces = None
face_maxiter = 120

for h in range(1, n):
    solver = FullProgramLPSolver(n=n, h=h, C=C, cache_tol=1e-4, cache_max=200_000, profile=True)

    interior = minimize_over_s_sorted(
        solver,
        n_random=interior_n_random,
        n_local_starts=interior_n_local_starts,
        seed=1,
        local_method="Powell",
        maxiter=interior_maxiter,
    )

    faces = minimize_over_faces(
        solver,
        seed=1,
        local_method="Powell",
        n_local_starts_per_face=face_local_starts_per_face,
        include_endpoints=face_include_endpoints,
        max_faces=face_max_faces,
        maxiter=face_maxiter,
    )

    print(
        f"\nRESULT n={n}, h={h}, C={C}\n"
        f"  face:     t={faces.t: .6g}, s={np.array2string(faces.s, precision=6, floatmode='fixed')}\n"
        f"  interior: t={interior.t: .6g}, s={np.array2string(interior.s, precision=6, floatmode='fixed')}"
    )

    solver.print_profile(label=f"(n={n}, h={h}, C={C})")


RESULT n=5, h=1, C=11.0
  face:     t= 3.46111, s=[0.685413 0.685413 0.685413 0.685413 0.685413]
  interior: t= 3.5266, s=[0.722945 0.714167 0.703571 0.680075 0.679380]

=== solve_lp profile (n=5, h=1, C=11.0) ===
calls:      6905
cache_hits: 3458 (50.1%)
misses:     3447 (49.9%)
build_eq:   0.628s
build_ub:   2.042s
linprog:    4.924s
total:      7.963s

RESULT n=5, h=2, C=11.0
  face:     t= 2.93413, s=[1.000000 0.483528 0.483528 0.483528 0.483528]
  interior: t= 3.03351, s=[1.000000 0.547716 0.547676 0.547579 0.390466]

=== solve_lp profile (n=5, h=2, C=11.0) ===
calls:      9101
cache_hits: 4809 (52.8%)
misses:     4292 (47.2%)
build_eq:   0.773s
build_ub:   2.635s
linprog:    6.851s
total:      10.726s

RESULT n=5, h=3, C=11.0
  face:     t= 2.45305, s=[0.777012 0.777012 0.777012 0.000000 0.000000]
  interior: t= 2.59231, s=[0.674862 0.669136 0.668955 0.250883 0.049515]

=== solve_lp profile (n=5, h=3, C=11.0) ===
calls:      9876
cache_hits: 5876 (59.5%)
misses:     4000 (40.5%)

In [2]:
# --------------------------
# NEW: outer optimization with s1 fixed to 1
# --------------------------

def minimize_over_s_sorted_s1_fixed(
    solver: FullProgramLPSolver,
    n_random: int = 120,
    n_local_starts: int = 3,
    seed: int = 0,
    local_method: str = "Powell",
    maxiter: int = 250,
) -> SearchResult:
    """
    Optimizes over s subject to:
        s1 = 1
        1 >= s1 >= s2 >= ... >= sn >= 0

    Parameterization:
        s1 = 1
        s2 = 1 * sigmoid(z1)
        s3 = s2 * sigmoid(z2)
        ...
        sn = s_{n-1} * sigmoid(z_{n-1})
    so monotonicity is enforced by construction.
    """
    n = solver.n
    rng = np.random.default_rng(seed)

    def z_to_s(z: np.ndarray) -> np.ndarray:
        z = np.asarray(z, dtype=float)
        u = sigmoid(z)  # in (0,1)
        s = np.empty(n, dtype=float)
        s[0] = 1.0
        prev = 1.0
        for i in range(1, n):
            prev = prev * u[i - 1]
            s[i] = prev
        return s

    def s_to_z(s: np.ndarray) -> np.ndarray:
        # Invert the mapping approximately:
        # u1 = s2
        # u_i = s_{i+1} / s_i  for i>=2
        s = np.asarray(s, dtype=float)
        u = np.empty(n - 1, dtype=float)
        u[0] = np.clip(s[1], 1e-6, 1 - 1e-6)
        for i in range(1, n - 1):
            denom = max(s[i], 1e-12)
            u[i] = np.clip(s[i + 1] / denom, 1e-6, 1 - 1e-6)
        return np.log(u / (1 - u))

    # ---- random scan to get a decent start
    best_t = float("inf")
    best_s = None

    for _ in range(n_random):
        # sample u in (0,1), build decreasing s via cumulative products
        u = np.clip(rng.random(n - 1), 1e-6, 1 - 1e-6)
        s = np.empty(n, dtype=float)
        s[0] = 1.0
        s[1:] = np.cumprod(u)
        val = solver.solve_lp(s, return_f=False)
        if val.ok and val.t < best_t:
            best_t = val.t
            best_s = s.copy()

    if best_s is None:
        best_s = np.linspace(1.0, 0.0, n)  # fallback, monotone with s1=1

    def obj(z):
        s = z_to_s(z)
        val = solver.solve_lp(s, return_f=False)
        return val.t if val.ok else 1e6

    starts = [s_to_z(best_s)]
    for _ in range(max(0, n_local_starts - 1)):
        u = np.clip(rng.random(n - 1), 1e-6, 1 - 1e-6)
        s0 = np.empty(n, dtype=float)
        s0[0] = 1.0
        s0[1:] = np.cumprod(u)
        starts.append(s_to_z(s0))

    best_z = None
    best_local = best_t

    for z0 in starts:
        res = minimize(obj, z0, method=local_method, options={"maxiter": maxiter, "disp": False})
        if float(res.fun) < best_local:
            best_local = float(res.fun)
            best_z = res.x.copy()

    s_star = z_to_s(best_z) if best_z is not None else best_s
    lp_star = solver.solve_lp(s_star, return_f=False)
    return SearchResult(lp_star.ok, lp_star.t, s_star, None, lp_star.msg)


# --------------------------
# OPTIONAL: face search variant with s1 fixed to 1
# --------------------------

def minimize_over_faces_s1_fixed(
    solver: FullProgramLPSolver,
    seed: int = 0,
    local_method: str = "Powell",
    n_local_starts_per_face: int = 1,
    include_endpoints: bool = True,
    max_faces: Optional[int] = None,
    maxiter: int = 120,
) -> SearchResult:
    """
    Same as minimize_over_faces, but restricts to faces with top_fixed=True (i.e., s1=1).
    """
    n = solver.n
    rng = np.random.default_rng(seed)

    faces_all = enumerate_faces(n, include_endpoints=include_endpoints)
    faces = [(tight_adj, True, bottom_fixed) for (tight_adj, top_fixed, bottom_fixed) in faces_all]

    if max_faces is not None:
        faces = faces[:max_faces]

    best = SearchResult(False, float("inf"), np.zeros(n), None, "no feasible face found")

    for (tight_adj, top_fixed, bottom_fixed) in faces:
        blocks = blocks_from_equalities(n, tight_adj)
        k = len(blocks)

        # If everything is pinned (degenerate) skip
        if k == 1 and top_fixed and bottom_fixed:
            continue

        dim = 0
        if not top_fixed:
            dim += 1
        dim += max(0, (k - 1) - (1 if bottom_fixed else 0))

        if dim == 0:
            v = np.zeros(k)
            if top_fixed:
                v[0] = 1.0
            if bottom_fixed:
                v[-1] = 0.0
            s = blocks_to_full_s(n, blocks, v)
            val = solver.solve_lp(s, return_f=False)
            if val.ok and val.t < best.t:
                best = SearchResult(True, val.t, s, (tight_adj, True, bottom_fixed), "OK")
            continue

        def obj(z):
            v = face_parameterization(z, k, top_fixed, bottom_fixed)
            s = blocks_to_full_s(n, blocks, v)
            val = solver.solve_lp(s, return_f=False)
            return val.t if val.ok else 1e6

        for _ in range(n_local_starts_per_face):
            z0 = rng.normal(size=dim)
            res = minimize(obj, z0, method=local_method, options={"maxiter": maxiter, "disp": False})
            if float(res.fun) < best.t:
                v = face_parameterization(res.x, k, top_fixed, bottom_fixed)
                s = blocks_to_full_s(n, blocks, v)
                best = SearchResult(True, float(res.fun), s, (tight_adj, True, bottom_fixed), "OK")

    return best

In [3]:
def best_block_then_zeros(
    solver: FullProgramLPSolver,
    *,
    min_k: int = 1,
    max_k: Optional[int] = None,
    n_grid: int = 401,
    refine_local: bool = True,
    local_method: str = "Powell",
    maxiter: int = 120,
) -> SearchResult:
    """
    Search over shapes of the form:
        s = [x, x, ..., x, 0, 0, ..., 0]
            ^---- k copies ----^  ^--- n-k zeros ---^
    where BOTH the block size k and the block value x are optimized.

    Constraints automatically satisfied:
        1 >= x >= 0 and the vector is nonincreasing.
    """
    n = solver.n
    if max_k is None:
        max_k = n
    min_k = int(min_k)
    max_k = int(max_k)
    assert 1 <= min_k <= max_k <= n

    best = SearchResult(False, float("inf"), np.zeros(n), None, "no feasible candidate")

    def make_s(k: int, x: float) -> np.ndarray:
        s = np.zeros(n, dtype=float)
        s[:k] = float(x)
        return s

    # grid over x for each k
    xs = np.linspace(0.0, 1.0, int(n_grid))
    for k in range(min_k, max_k + 1):
        for x in xs:
            s = make_s(k, x)
            val = solver.solve_lp(s, return_f=False)
            if val.ok and val.t < best.t:
                best = SearchResult(True, val.t, s, face=("block_then_zeros", k, float(x)), msg=val.msg)

    if not (refine_local and best.ok):
        return best

    # local refinement in x for the best k (1D)
    tag, k_best, x_best = best.face
    assert tag == "block_then_zeros"

    def obj_x(z):
        x = sigmoid(z[0])  # (0,1)
        s = make_s(k_best, x)
        v = solver.solve_lp(s, return_f=False)
        return v.t if v.ok else 1e6

    # init z from x_best
    x0 = np.clip(float(x_best), 1e-6, 1 - 1e-6)
    z0 = np.array([np.log(x0 / (1 - x0))], dtype=float)

    res = minimize(obj_x, z0, method=local_method, options={"maxiter": maxiter, "disp": False})
    x_ref = float(sigmoid(res.x[0]))
    s_ref = make_s(k_best, x_ref)
    v_ref = solver.solve_lp(s_ref, return_f=False)

    if v_ref.ok and v_ref.t < best.t:
        best = SearchResult(True, v_ref.t, s_ref, face=("block_then_zeros", k_best, x_ref), msg=v_ref.msg)

    return best

In [18]:
n = 5
h = 2
C = 14.75
solver = FullProgramLPSolver(n=n, h=h, C=C, cache_tol=1e-4, cache_max=200_000, profile=True)

# If you also want the face-search version with s1=1:
res_face = minimize_over_faces_s1_fixed(
    solver,
    seed=1,
    local_method="Powell",
    n_local_starts_per_face=1,
    include_endpoints=True,
    maxiter=120,
)
print(f"FACE RESULT s1 fixed: ok={res_face.ok}, t={res_face.t:.6g}, s={np.array2string(res_face.s, precision=6, floatmode='fixed')}, msg={res_face.msg}")

res_block0 = best_block_then_zeros(
    solver,
    min_k=1,
    max_k=solver.n,
    n_grid=801,          # finer grid if you want
    refine_local=True,   # refine x for best k
    local_method="Powell",
    maxiter=80,
)
print(
    f"BLOCK-THEN-ZEROS BEST: ok={res_block0.ok}, t={res_block0.t:.6g}, "
    f"s={np.array2string(res_block0.s, precision=6, floatmode='fixed')}"
)

FACE RESULT s1 fixed: ok=True, t=3.15139, s=[1.000000 0.537828 0.537828 0.537828 0.537828], msg=OK
BLOCK-THEN-ZEROS BEST: ok=True, t=3.15104, s=[0.600000 0.600000 0.600000 0.600000 0.600000]


In [7]:
n = 6
# C = 11.0

results = []
for C in [23.]:
# for C in [11.]:
    for h in range(1, n):
        print(f"n={n}, h={h}, C={C}")

        solver = FullProgramLPSolver(
            n=n,
            h=h,
            C=C,
            cache_tol=1e-4,
            cache_max=200_000,
            profile=True,
        )

        # ---- Face search with s1 fixed
        res_face = minimize_over_faces_s1_fixed(
            solver,
            seed=1,
            local_method="Powell",
            n_local_starts_per_face=1,
            include_endpoints=True,
            maxiter=120,
        )

        # ---- Block-then-zeros search
        res_block0 = best_block_then_zeros(
            solver,
            min_k=1,
            max_k=solver.n,
            n_grid=801,
            refine_local=True,
            local_method="Powell",
            maxiter=80,
        )

        # ---- Decide winner
        t_face = res_face.t if res_face.ok else float("inf")
        t_block = res_block0.t if res_block0.ok else float("inf")

        face_star = ""
        block_star = ""
        if abs(t_face - t_block) < 1e-9:
            face_star = block_star = "(*)"
        elif t_face < t_block:
            face_star = "(*)"
        else:
            block_star = "(*)"

        print(
            f"  {face_star}FACE RESULT (s1 fixed):"
            f"ok={res_face.ok}, "
            f"t={res_face.t:.6g}, "
            f"s={np.array2string(res_face.s, precision=3, floatmode='fixed')}"
        )

        print(
            f"  {block_star}BLOCK-THEN-ZEROS BEST:"
            f"ok={res_block0.ok}, "
            f"t={res_block0.t:.6g}, "
            f"s={np.array2string(res_block0.s, precision=3, floatmode='fixed')}"
        )

        results.append({
            "h": h,
            "face_ok": res_face.ok,
            "face_t": res_face.t,
            "face_s": res_face.s,
            "block_ok": res_block0.ok,
            "block_t": res_block0.t,
            "block_s": res_block0.s,
            "block_face": res_block0.face,
            "winner": "face" if t_face < t_block else ("block" if t_block < t_face else "tie"),
        })



n=6, h=1, C=23.0
  FACE RESULT (s1 fixed):ok=True, t=4.92865, s=[1.000 0.786 0.786 0.786 0.786 0.786]
  (*)BLOCK-THEN-ZEROS BEST:ok=True, t=4.76034, s=[0.793 0.793 0.793 0.793 0.793 0.793]
n=6, h=2, C=23.0
  FACE RESULT (s1 fixed):ok=True, t=3.9331, s=[1.000 0.587 0.587 0.587 0.587 0.587]
  (*)BLOCK-THEN-ZEROS BEST:ok=True, t=3.81693, s=[0.608 0.608 0.608 0.608 0.608 0.608]
n=6, h=3, C=23.0
  FACE RESULT (s1 fixed):ok=True, t=3.36587, s=[1.000 0.473 0.473 0.473 0.473 0.473]
  (*)BLOCK-THEN-ZEROS BEST:ok=True, t=3.34681, s=[0.642 0.642 0.642 0.642 0.642 0.000]
n=6, h=4, C=23.0
  FACE RESULT (s1 fixed):ok=True, t=2.76019, s=[1.000 0.880 0.880 0.000 0.000 0.000]
  (*)BLOCK-THEN-ZEROS BEST:ok=True, t=2.68632, s=[0.883 0.883 0.883 0.000 0.000 0.000]
n=6, h=5, C=23.0
  (*)FACE RESULT (s1 fixed):ok=True, t=1.91671, s=[1.000 0.917 0.000 0.000 0.000 0.000]
  BLOCK-THEN-ZEROS BEST:ok=True, t=1.95652, s=[0.957 0.957 0.000 0.000 0.000 0.000]


In [ ]:
n = 8
# C = 11.0

results = []
for C in [11., 13., 15., 17.]:
# for C in [11.]:
    for h in range(1, n):
        print(f"n={n}, h={h}, C={C}")

        solver = FullProgramLPSolver(
            n=n,
            h=h,
            C=C,
            cache_tol=1e-4,
            cache_max=200_000,
            profile=True,
        )

        # ---- Face search with s1 fixed
        res_face = minimize_over_faces_s1_fixed(
            solver,
            seed=1,
            local_method="Powell",
            n_local_starts_per_face=1,
            include_endpoints=True,
            maxiter=120,
        )

        # ---- Block-then-zeros search
        res_block0 = best_block_then_zeros(
            solver,
            min_k=1,
            max_k=solver.n,
            n_grid=801,
            refine_local=True,
            local_method="Powell",
            maxiter=80,
        )

        # ---- Decide winner
        t_face = res_face.t if res_face.ok else float("inf")
        t_block = res_block0.t if res_block0.ok else float("inf")

        face_star = ""
        block_star = ""
        if abs(t_face - t_block) < 1e-9:
            face_star = block_star = "(*)"
        elif t_face < t_block:
            face_star = "(*)"
        else:
            block_star = "(*)"

        print(
            f"  {face_star}FACE RESULT (s1 fixed):"
            f"ok={res_face.ok}, "
            f"t={res_face.t:.6g}, "
            f"s={np.array2string(res_face.s, precision=3, floatmode='fixed')}"
        )

        print(
            f"  {block_star}BLOCK-THEN-ZEROS BEST:"
            f"ok={res_block0.ok}, "
            f"t={res_block0.t:.6g}, "
            f"s={np.array2string(res_block0.s, precision=3, floatmode='fixed')}"
        )

        results.append({
            "h": h,
            "face_ok": res_face.ok,
            "face_t": res_face.t,
            "face_s": res_face.s,
            "block_ok": res_block0.ok,
            "block_t": res_block0.t,
            "block_s": res_block0.s,
            "block_face": res_block0.face,
            "winner": "face" if t_face < t_block else ("block" if t_block < t_face else "tie"),
        })



n=8, h=1, C=11.0
  FACE RESULT (s1 fixed):ok=True, t=4.88927, s=[1.000 0.556 0.556 0.556 0.556 0.556 0.556 0.556]
  (*)BLOCK-THEN-ZEROS BEST:ok=True, t=4.63809, s=[0.578 0.578 0.578 0.578 0.578 0.578 0.578 0.578]
n=8, h=2, C=11.0
  FACE RESULT (s1 fixed):ok=True, t=3.85631, s=[1.000 0.408 0.408 0.408 0.408 0.408 0.408 0.408]
  (*)BLOCK-THEN-ZEROS BEST:ok=True, t=3.85413, s=[0.424 0.424 0.424 0.424 0.424 0.424 0.424 0.424]
n=8, h=3, C=11.0
